# 2021-11-29 Cython implementation for ncut

Load in test data

In [2]:
model_file='/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/05_model_output/01_group/bsc_python_cluster_100_ngram_34_min_100.obj'

data_file='/home/hubert/DPhil_Studies/2021-04_Study_A_Diffusion/data/02_intermediate/01_group/user_count_mat_ngram_34.obj'

%load_ext Cython

In [3]:
import pickle
with open(data_file, 'rb') as f:
    data = pickle.load(f)
with open(model_file, 'rb') as f:
    model = pickle.load(f)

Python

In [4]:
import numpy as np
def ncut_cluster(cocluster, csr, i):

    rows, cols = cocluster.get_indices(i)
    if not (np.any(rows) and np.any(cols)):
        # return sys.float_info.max
        return 0
    row_complement = np.nonzero(np.logical_not(cocluster.rows_[i]))[0]
    col_complement = np.nonzero(np.logical_not(cocluster.columns_[i]))[0]
    # Note: the following is identical to X[rows[:, np.newaxis],
    # cols].sum() but much faster in scipy <= 0.16
    weight = csr[rows][:, cols].sum()
    # weight = csr[rows[:, np.newaxis],cols].sum()
    cut = csr[row_complement][:, cols].sum() + csr[rows][:, col_complement].sum()
    return cut / weight

In [ ]:
%timeit ncut_cluster(model, data, 0)

Cython

In [21]:
model.n_clusters

100

In [5]:
rows, cols = model.get_indices(0)

In [6]:
x = model.rows_[0]
y = model.columns_[0]
x = np.nonzero(np.logical_not(x))[0]
y = np.nonzero(np.logical_not(y))[0]

In [16]:
data[rows][:,cols]

<3318x168177 sparse matrix of type '<class 'numpy.int64'>'
	with 147615 stored elements in Compressed Sparse Column format>

In [20]:
%%cython -a
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

import numpy as np
cimport numpy as np
cimport cython
from scipy.sparse import csr_matrix
np.import_array()

@cython.boundscheck(False)
@cython.wraparound(False)
def test(np.ndarray rows, np.ndarray cols, np.ndarray row_c, np.ndarray col_c, m):

    # cdef nd.array[np.int64_t, ndim=1] rows = rows
    # cdef nd.array[np.int64_t, ndim=1] cols = cols
    # cdef nd.array[np.int64_t, ndim=1] row_c = row_c
    # cdef nd.array[np.int64_t, ndim=1] col_c = col_c

    weight = m[rows][:,cols].sum()
    cut = m[row_c][:,cols].sum() + m[rows][:,col_c].sum()
    result = cut/weight
    return cut/weight


In file included from /home/hubert/.local/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944,
                 from /home/hubert/.local/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/hubert/.local/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/hubert/.cache/ipython/cython/_cython_magic_35b396946a7721bc295078a86f600867.c:656:
/home/hubert/.local/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [19]:
test(rows, cols, x, y, data)

1.319064322131368

In [ ]:
%%cython

@cython.boundscheck(False)
@cython.wraparound(False)
def ncut_cluster_cython(cocluster, csr, i):

    rows, cols = cocluster.get_indices(i)
    if not (np.any(rows) and np.any(cols)):
        # return sys.float_info.max
        return 0

    model_rows = model.rows_[i]
    model_cols = model.columns_[i]

    weight = csr[rows][:,cols].sum()

    row_complement = np